Load dataset

In [ ]:
import pandas as pd
import csv

In [ ]:
class DataSet():
    def __init__(self, name="train", path="dataset"):
        pass

    def read(self,bodies,stances):
        pass
    
    def print(self):
        print(self.data.head())

In [ ]:
training = DataSet()
test = DataSet(name="competition_test")

In [ ]:
print(training.data.head())

Cleaning the text and tokenizing

In [ ]:
import os
import re
import nltk
import numpy as np
from sklearn import feature_extraction
from tqdm import tqdm

_wnl = nltk.WordNetLemmatizer() 

def normalize_word(w):
    pass

def get_tokenized_lemmas(s):
    pass

def clean(s):
    # Cleans a string: Lowercasing, trimming, removing non-alphanumeric
    pass

def remove_stopwords(l):
    # Removes stopwords from a list of tokens
    pass

def get_clean_tokens(s):
    s = clean(s)
    tokens = get_tokenized_lemmas(s)
    clean_tokens = remove_stopwords(tokens)
    return clean_tokens

In [ ]:
training.data['Headline tokens'] = #...

In [ ]:
print(training.data.head())

In [ ]:
# Reload the dataframe because it takes a long time to transform the body :-) 
training.data = pd.read_pickle("trainingdatatokens.pkl")

In [ ]:
print(training.data.head())

In [ ]:
# view most common words
from collections import Counter

min_len = 2 #tunable parameter

count_words = Counter()
for tokens in training.data['articleBody tokens'].values:
    # ...?

In [ ]:
count_words.most_common(10)

Vectorize: from text to a vector space

In [ ]:
vocabulary = #...

In [ ]:
listed_docs = #...
listed_docs2 = #...

Vectorize - fit: Fit vocabulary means generate the vector space for the words

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(2,2), min_df=5)
#
#

Vectorize - transform: Represent a document on the vector space that was fit previously

In [ ]:
training.data['tfidf headlines'] = #...

In [ ]:
print(training.data.head())

Load data just to make it less heavy

In [ ]:
training.data = pd.read_pickle("data_tfidf.pkl")

How does the vector of a document looks like?

In [ ]:
#

In [ ]:
#
#

Measuring similarity between two documents

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
a = vectorizer.transform([listed_docs[0]])
b = vectorizer.transform([listed_docs[200]])
print(cosine_similarity(a,b))

Find the headline closest to the first body

In [ ]:
# find the headline closest to the first body:
#
#
#
#
#
#

In [ ]:
print("Similarity: %f" %sim_old)
print("Most similar: %s" %listed_docs2[most_similar_indx])
print("Match for: %s" %listed_docs[0])

# Reintroduce the task

Ok, so let's do some fake news detection.

Given a headline and a body of text, we want to say whether these two are:

* unrelated
* agree with each other
* disagree with each other
* discuss each other


# Features ?

First, we have to generate features.

Let's check:

* Word overlap (between headline and body)
* Refuting features: words which are refuting
* Polarity features: words which contain polarity
* n-gram
* char-grams
* co-occurences

In [ ]:
def word_overlap_features(headlines, bodies):
    # Computes the percentage of overlap between the headline and the body of text (numerical)
    pass


In [ ]:
word_overlap_features(list(training.data['Headline'].values[0:10]), list(training.data['articleBody'].values[0:10]))

In [ ]:
def refuting_features(headlines, bodies):
    _refuting_words = [
        'fake',
        'fraud',
        'hoax',
        'false',
        'deny', 'denies',
        'not',
        'despite',
        'nope',
        'doubt', 'doubts',
        'bogus',
        'debunk',
        'pranks',
        'retract'
    ]
    # pass
    return X

In [ ]:
refuting_features(list(training.data['Headline'].values)[0:1], list(training.data['articleBody'].values)[0:1])

In [ ]:
def polarity_features(headlines, bodies):
    _refuting_words = [
        'fake',
        'fraud',
        'hoax',
        'false',
        'deny', 'denies',
        'not',
        'despite',
        'nope',
        'doubt', 'doubts',
        'bogus',
        'debunk',
        'pranks',
        'retract'
    ]

    def calculate_polarity(text):
        tokens = get_tokenized_lemmas(text)
        return sum([t in _refuting_words for t in tokens]) % 2
    
    X = []
    #pass
    return np.array(X)

In [ ]:
polarity_features(list(training.data['Headline'].values)[0:10], list(training.data['articleBody'].values)[0:10])

Some other features... 

In [ ]:
def ngrams(input, n):
    input = input.split(' ')
    output = []
    for i in range(len(input) - n + 1):
        output.append(input[i:i + n])
    return output

def chargrams(input, n):
    output = []
    for i in range(len(input) - n + 1):
        output.append(input[i:i + n])
    return output

def append_chargrams(features, text_headline, text_body, size):
    grams = [' '.join(x) for x in chargrams(" ".join(remove_stopwords(text_headline.split())), size)]
    grams_hits = 0
    grams_early_hits = 0
    grams_first_hits = 0
    for gram in grams:
        if gram in text_body:
            grams_hits += 1
        if gram in text_body[:255]:
            grams_early_hits += 1
        if gram in text_body[:100]:
            grams_first_hits += 1
    features.append(grams_hits)
    features.append(grams_early_hits)
    features.append(grams_first_hits)
    return features

def append_ngrams(features, text_headline, text_body, size):
    grams = [' '.join(x) for x in ngrams(text_headline, size)]
    grams_hits = 0
    grams_early_hits = 0
    for gram in grams:
        if gram in text_body:
            grams_hits += 1
        if gram in text_body[:255]:
            grams_early_hits += 1
    features.append(grams_hits)
    features.append(grams_early_hits)
    return features

def hand_features(headlines, bodies):
    def binary_co_occurence(headline, body):
        # Count how many times a token in the title
        # appears in the body text.
        bin_count = 0
        bin_count_early = 0
        for headline_token in clean(headline).split(" "):
            if headline_token in clean(body):
                bin_count += 1
            if headline_token in clean(body)[:255]:
                bin_count_early += 1
        return [bin_count, bin_count_early]

    def binary_co_occurence_stops(headline, body):
        # Count how many times a token in the title
        # appears in the body text. Stopwords in the title
        # are ignored.
        bin_count = 0
        bin_count_early = 0
        for headline_token in remove_stopwords(clean(headline).split(" ")):
            if headline_token in clean(body):
                bin_count += 1
                bin_count_early += 1
        return [bin_count, bin_count_early]

    def count_grams(headline, body):
        # Count how many times an n-gram of the title
        # appears in the entire body, and intro paragraph
        clean_body = clean(body)
        clean_headline = clean(headline)
        features = []
        features = append_chargrams(features, clean_headline, clean_body, 2)
        features = append_chargrams(features, clean_headline, clean_body, 8)
        features = append_chargrams(features, clean_headline, clean_body, 4)
        features = append_chargrams(features, clean_headline, clean_body, 16)
        features = append_ngrams(features, clean_headline, clean_body, 2)
        features = append_ngrams(features, clean_headline, clean_body, 3)
        features = append_ngrams(features, clean_headline, clean_body, 4)
        features = append_ngrams(features, clean_headline, clean_body, 5)
        features = append_ngrams(features, clean_headline, clean_body, 6)
        return features

    X = []
    for headline, body in zip(headlines, bodies):
        X.append(binary_co_occurence(headline, body)
                 + binary_co_occurence_stops(headline, body)
                 + count_grams(headline, body))

    return X

In [ ]:
hand_features(list(training.data['Headline'].values)[0:1], list(training.data['articleBody'].values)[0:1])

In [ ]:
def gen_or_load_feats(feat_fn, headlines, bodies, feature_file):
    if not os.path.isfile(feature_file):
        feats = feat_fn(headlines, bodies)
        np.save(feature_file, feats)

    return np.load(feature_file)

In [ ]:
LABELSINT = {'agree':0, 'disagree':1, 'discuss':2, 'unrelated':3}

def generate_features(dataset,name):
    h, b, y = [],[],[]

    y = [LABELSINT[label] for label in list(dataset['Stance'].values)]
    h = list(dataset['Headline'].values)
    b = list(dataset['articleBody'].values)

    X_overlap  = #
    X_refuting = #
    X_polarity = #
    X_hand = #
    
    X = np.c_[X_hand, X_polarity, X_refuting, X_overlap]
    return X,y

# Classification
Now, we choose our "favourite" classification algorithm

In [ ]:
from utils.dependencies import *

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

LABELS = ['agree', 'disagree', 'discuss', 'unrelated']
LABELS_RELATED = ['unrelated','related']
RELATED = LABELS[0:3]

d = DataSet("train")
folds,hold_out = kfold_split(d,n_folds=10)
fold_stances, hold_out_stances = get_stances_for_folds(d,folds,hold_out)

# Load the competition dataset
competition_dataset = DataSet("competition_test")
X_competition, y_competition = generate_features(competition_dataset.data, "competition")

training = DataSet()
test = DataSet(name="competition_test")
Xs = dict()
ys = dict()

# Load/Precompute all features now
#
#
#

best_score = 0
best_fold = None

# Classifier for each fold
for fold in fold_stances:
    ids = list(range(len(folds)))
    del ids[fold]

    X_train = np.vstack(tuple([Xs[i] for i in ids]))
    y_train = np.hstack(tuple([ys[i] for i in ids]))

    X_test = Xs[fold]
    y_test = ys[fold]

    clf = # 
    clf.fit(X_train, y_train)

    predicted = [LABELS[int(a)] for a in clf.predict(X_test)]
    actual = [LABELS[int(a)] for a in y_test]

    fold_score, _ = score_submission(actual, predicted)
    max_fold_score, _ = score_submission(actual, actual)

    score = fold_score/float(max_fold_score)

    print("Score for fold "+ str(fold) + " was - " + str(score))
    if score > best_score:
        best_score = score
        best_fold = clf


#Run on Holdout set and report the final score on the holdout set
predicted = [LABELS[int(a)] for a in best_fold.predict(X_holdout)]
actual = [LABELS[int(a)] for a in y_holdout]

print("Scores on the dev set")
report_score(actual,predicted)
print("")
print("")

#Run on competition dataset
predicted = [LABELS[int(a)] for a in best_fold.predict(X_competition)]
actual = [LABELS[int(a)] for a in y_competition]

print("Scores on the test set")
report_score(actual,predicted)

In [ ]:
#Run on competition dataset
predicted = [LABELS[int(a)] for a in best_fold.predict(X_competition)]
actual = [LABELS[int(a)] for a in y_competition]

print("Scores on the test set")
report_score(actual,predicted)